In [1]:
!pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 4.0 MB/s eta 0:00:00a 0:00:01


In [41]:
import os
import pandas as pd
import nltk
import re

In [4]:
with open('scraped_articles/Netclan20241017.txt', 'r') as file:
    text = file.read()

text

'AI and ML-Based YouTube Analytics and Content Creation Tool for Optimizing Subscriber Engagement and Content Strategy - Blackcoffer Insights\nClient Background\n\nClient:A leading IT & tech firm in the USA\n\nIndustry Type:IT\n\nProducts & Services:IT Consulting, IT Support, SaaS, Marketing Strategy\n\nOrganization Size:10+\n\nThe Problem\n\nBuilding AI and ML based YouTube analytics and content creation tool that will help youtuber to understand their subscriber’s watching behaviour, help them in content research, creation and publication.\n\nOur Solution\n\nCreated a MERN stack web application and integrated AI models to helps youtuber to generated titles, descriptions, tags, hashtags, captions etc. Help them to check thumbnail quality, analysis on the videos using video auditor tool, analysis on comments using sentiments analysis, help to under their subscribers using churn predication AI model.\n\nSolution Architecture\n\nDeliverables\n\nWeb Applications\n\nSupports\n\nMaintenance

In [13]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')

# Creating a set of stop words that is removed from the entire corpus of text
stop_words = set(stopwords.words('english'))

with open('positive-words.txt', 'r') as file:
    pos_words = file.read().splitlines()

with open('negative-words.txt', 'r') as file:
    neg_words = file.read().splitlines()

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/anuragprasad/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [15]:
keys = ['positive', 'negative']
values = [pos_words, neg_words]
master_dict = dict(zip(keys,values))

In [35]:
data = pd.read_csv('Input.xlsx - Sheet1.csv')

In [36]:
content = []
for file in os.listdir('scraped_articles'):

    with open(f'scraped_articles/{file}', 'r') as file:
        article = file.read()
    content.append(article)
data['content'] = content

In [37]:
def give_scores(data):
    for i in range(len(data)):
        text_tokens = word_tokenize(data.iloc[i]['content'])
        filtered_words = [w for w in text_tokens if not w in stop_words]
        filtered_text = ' '.join(filtered_words)

        pos_score = 0
        neg_score = 0
        for word in filtered_text.split(' '):
            if word in pos_words:
                pos_score += 1
            elif word in neg_words:
                neg_score += 1

        neg_score *= -1
        data.at[i, 'POSITIVE_SCORE'] = pos_score
        data.at[i, 'NEGATIVE_SCORE'] = neg_score
        
        total_score = pos_score + neg_score
        data.at[i, 'POLARITY_SCORE'] = (pos_score - neg_score)/(total_score + 0.000001)
        data.at[i, 'SUBJECTIVITY_SCORE'] = total_score/(len(filtered_words) + 0.000001)

In [42]:
# This finds all the attributes related to fog index
def fog_index(data):
    for i in range(len(data)):
        article_text = data.iloc[i]['CONTENT']
        text_tokens = word_tokenize(article_text)
        text_words = [word for word in text_tokens if word.strip() and word.isalpha()]
        sentences = re.split(r'[.!?]+', article_text)
        sentences = [s for s in sentences if s.strip()]


        complex_words = [word for word in text_tokens if textstat]

        data.at[i,'AVG_SENTENCE_LENGTH'] = len(text_words)/len(sentences)

In [38]:
give_scores(data = data)

In [40]:
data.head()

,URL_ID,URL,content,POSITIVE_SCORE,NEGATIVE_SCORE,POLARITY_SCORE,SUBJECTIVITY_SCORE
0,Netclan20241017,https://insights.blackcoffer.com/ai-and-ml-bas...,ROAS Dashboard for Campaign-Wise Google Ads Bu...,9.0,-7.0,7.999996,0.004049
1,Netclan20241018,https://insights.blackcoffer.com/enhancing-fro...,Analyzing the Impact of Positive Emotions and ...,11.0,-3.0,1.750000,0.018018
2,Netclan20241019,https://insights.blackcoffer.com/roas-dashboar...,Enhancing Front-End Features and Functionality...,12.0,-7.0,3.799999,0.007622
3,Netclan20241020,https://insights.blackcoffer.com/efficient-pro...,Google Local Service Ads Missed Calls and Mess...,0.0,0.0,0.000000,0.000000
4,Netclan20241021,https://insights.blackcoffer.com/development-o...,Splitting of Songs into its Vocals and Instrum...,0.0,0.0,0.000000,0.000000


In [57]:
df = pd.DataFrame(columns = ['POSITIVE SCORE', 'NEGATIVE SCORE', 'POLARITY SCORE', 
                             'SUBJECTIVITY SCORE', 'AVG SENTENCE LENGTH', 
                             'PERCENTAGE OF COMPLEX WORDS', 'FOG INDEX', 
                             ' AVG NUMBER OF WORDS PER SENTENCE',
                              'COMPLEX WORD COUNT', 'WORD COUNT',
                              'SYLLABLE PER WORD', 'PERSONAL PRONOUNS',
                              'AVG WORD LENGTH'])